# Demo Notebook how to run models on static mouse datasets

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2 

In [2]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True

name = 'iclr' #"test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

In [3]:
import torch
import numpy as np
import pickle 
import pandas as pd
from collections import OrderedDict, Iterable
import matplotlib.pyplot as plt

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder
from nnfabrik.utility.hypersearch import Bayesian

from nnsysident.tables.experiments import *
from nnsysident.tables.bayesian import *
from nnsysident.datasets.mouse_loaders import static_shared_loaders
from nnsysident.datasets.mouse_loaders import static_loaders
from nnsysident.datasets.mouse_loaders import static_loader

<ipython-input-3-466f6e3fc979>:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import OrderedDict, Iterable


Connecting konstantin@134.2.168.16:3306
Schema name: konstantin_nnsysident_iclr


In [4]:
TrainedModelBayesianTransfer()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnsysident.models.models.se2d_fullgaussian2d,ec098d02dbfde45a254be2be3b06aad1,nnsysident.datasets.mouse_loaders.static_shared_loaders,58b884e78ae93d09cb49f6a22ee3a985,nnsysident.training.trainers.standard_trainer,b5b26ea7fd9075bc859c18add25efa59,1,Bayesian optimization of Hyper params..Bayesian optimization of Hyper params..Bayesian optimization of Hyper params.,0.333055,=BLOB=,kklurz,2020-08-21 16:10:15


In [5]:
for neuron_n in [50]: # 500, 1000,
    for image_n in [1000]: # [ 50, 200, 500, 1000, 2500, 4399]
        # try:
        paths = ['data/static22564-2-12-preproc0.zip',
                 'data/static22564-2-13-preproc0.zip',
                 'data/static22564-3-8-preproc0.zip',
                 'data/static22564-3-12-preproc0.zip']

        dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'
        dataset_config = dict(
            paths=paths,
            batch_size=64,
            multi_match_n=neuron_n,
            multi_match_base_seed=1,
            image_n=image_n,
            image_base_seed=1

        )
        dataset_config_auto = dict()
        print(dataset_config)

        model_fn = 'nnsysident.models.models.se2d_fullgaussian2d'
        model_config = {#"share_features": True,
                        #"share_transform":True,
                        "init_mu_range": 0.55,
                        "init_sigma": 0.4,
                        'input_kern': 15,
                        'hidden_kern': 13,
                        'gamma_input': 1.,
                        'grid_mean_predictor': {'type': 'cortex', 'input_dimensions': 2, 'hidden_layers': 0, 'hidden_features': 0, 'final_tanh': False},
                        'transfer_state_dict': 'models/24cb5dd8d8c63765b6435392cd14a29e.pth.tar'
                        
        }
        print(model_fn)
        print(model_config)
        model_config_auto = dict(
            gamma_readout={"type": "range", "bounds": [1e-5, 1e3], "log_scale": True},
            share_features={"type": "choice", "values": [True, False]},
            share_transform={"type": "choice", "values": [True, False]},
            # init_transform_scale={"type": "range", "bounds": [0.1, 0.7]},
        )


        trainer_fn = 'nnsysident.training.trainers.standard_trainer'
        trainer_config = dict(detach_core=True)
        trainer_config_auto = dict(
                      )

        autobayes = Bayesian(dataset_fn, dataset_config, dataset_config_auto,
                             model_fn, model_config, model_config_auto,
                             trainer_fn, trainer_config, trainer_config_auto, architect="kklurz",
                             trained_model_table='nnsysident.tables.bayesian.TrainedModelBayesianTransfer', total_trials=1)

        best_parameters, _, _, _ = autobayes.run()

[INFO 08-21 16:13:12] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 08-21 16:13:12] ax.service.managed_loop: Started full optimization with 1 steps.
[INFO 08-21 16:13:12] ax.service.managed_loop: Running optimization trial 1...


{'paths': ['data/static22564-2-12-preproc0.zip', 'data/static22564-2-13-preproc0.zip', 'data/static22564-3-8-preproc0.zip', 'data/static22564-3-12-preproc0.zip'], 'batch_size': 64, 'multi_match_n': 50, 'multi_match_base_seed': 1, 'image_n': 1000, 'image_base_seed': 1}
nnsysident.models.models.se2d_fullgaussian2d
{'init_mu_range': 0.55, 'init_sigma': 0.4, 'input_kern': 15, 'hidden_kern': 13, 'gamma_input': 1.0, 'grid_mean_predictor': {'type': 'cortex', 'input_dimensions': 2, 'hidden_layers': 0, 'hidden_features': 0, 'final_tanh': False}, 'transfer_state_dict': 'models/24cb5dd8d8c63765b6435392cd14a29e.pth.tar'}
data/static22564-2-12-preproc0 exists already. Not unpacking data/static22564-2-12-preproc0.zip
data/static22564-2-13-preproc0 exists already. Not unpacking data/static22564-2-13-preproc0.zip
data/static22564-3-8-preproc0 exists already. Not unpacking data/static22564-3-8-preproc0.zip
data/static22564-3-12-preproc0 exists already. Not unpacking data/static22564-3-12-preproc0.zip
d

/src/nnsysident/nnsysident/models/models.py:144: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.

Epoch 1: 100%|██████████| 64/64 [00:04<00:00, 12.83it/s]


[001|01/05] -/-> -0.001694628270342946


Epoch 2: 100%|██████████| 64/64 [00:01<00:00, 52.42it/s]


...

[075|03/05] -/-> 0.339161217212677


Epoch 76: 100%|██████████| 64/64 [00:01<00:00, 58.53it/s]


[076|04/05] -/-> 0.33975815773010254


Epoch 77: 100%|██████████| 64/64 [00:01<00:00, 60.20it/s]


[077|05/05] -/-> 0.3391374349594116
Restoring best model after lr decay! 0.339137 ---> 0.340573
Restoring best model! 0.340573 ---> 0.340573


# Get Dataloader

In [ ]:
data = pd.DataFrame((TrainedModelBayesian() * 
                     ModelBayesian * 
                     DatasetBayesian * 
                     Trainer & 
                     'model_fn = "nnsysident.models.models.se2d_fullgaussian2d"' & 
                     'trainer_hash = "3c6008284286683e7ce19e9e1269f507"').fetch())
data = pd.concat([data, data['dataset_config'].apply(pd.Series)], axis = 1)#.drop('dataset_config', axis = 1)
data = pd.concat([data, data['model_config'].apply(pd.Series)], axis = 1)#.drop('model_config', axis = 1)
data = pd.concat([data, data['trainer_config'].apply(pd.Series)], axis = 1)#.drop('model_config', axis = 1)

In [ ]:
import hiplot as hip
from nnsysident.tables.bayesian import *

one_exp_h = data.set_index('score', drop=False).copy()
cols = ['score', 'gamma_readout', 'share_features', 'share_transform', 'multi_match_n', 'image_n']
hip.Experiment.from_dataframe(one_exp_h[cols]).display(force_full_width=False)

In [ ]:
# change path here
paths = ['data/static22564-2-12-preproc0.zip',
                     'data/static22564-2-13-preproc0.zip',
                     'data/static22564-3-8-preproc0.zip',
                     'data/static22564-3-12-preproc0.zip']

#paths = ['data/static22564-2-12-preproc0.zip']
dataset_fn = 'nnsysident.datasets.mouse_loaders.static_shared_loaders'
dataset_config = dict(
    paths=paths,
    batch_size=64,
    seed=1,
    #image_n=50,
    #image_base_seed=1,
    multi_match_n=972,
    multi_match_base_seed=1,
    exclude_multi_match_n = 3625,

)

dataloaders = builder.get_data(dataset_fn, dataset_config)

# Get Model

### The New gaussian readout: change gauss_type for the different modes

In [ ]:
model_fn = 'nnsysident.models.models.se2d_spatialxfeaturelinear'

model_config = { 'gamma_input': 1.0,
                 'input_kern': 15,
                 'hidden_kern': 13,
                 'gamma_readout': 0.011027209808690062}


model = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)
model_real = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)
model_start = builder.get_model(model_fn=model_fn, model_config=model_config, dataloaders=dataloaders, seed=1)

In [ ]:
core_dict = torch.load('f8acb861ca08a5baa6a2824ca33f51aa.pth.tar') # score: 0.354544

def get_grids(my_model):
    grids = {}
    for key, readout in my_model.readout.items():
        grid = readout.grid.squeeze().cpu().data.numpy()
        grids[key] = grid
    return grids

list_of_load = [] #'scales', '_features' ,'mu_transform.0.weight', 'mu_transform.0.bias'
list_of_detach = []



model_real.load_state_dict(core_dict, strict=False)
real_grids = get_grids(model_real)

remove=[]
keep=[]
for key in core_dict.keys():
    name = '.'.join(key.split('.')[2:])
    if key.split('.')[0] == 'readout': 
        if not np.isin(name, list_of_load):
            print('Not loading:    {}'.format(key))
            remove.append(key)
        else:
            keep.append(key)
for key in keep:
    print('Loading:  {}'.format(key))

for k in remove: del core_dict[k]
model_start.load_state_dict(core_dict, strict=False)
start_grids = get_grids(model_start)



model.load_state_dict(core_dict, strict=False)

for param in model.named_parameters():
    name = '.'.join(param[0].split('.')[2:])
    if param[0].split('.')[0] == 'readout':
        if np.isin(name, list_of_detach):
            print('detaching:    {}'.format(param[0]))
            param[1].requires_grad = False

# Get Trainer

In [ ]:
trainer_fn = 'nnsysident.training.trainers.standard_trainer'
trainer_config = dict(track_training=True) #detach_core=True, 
trainer = builder.get_trainer(trainer_fn, trainer_config)

# Run Training

In [ ]:
score, output, model_state = trainer(model=model, dataloaders=dataloaders, seed=1)

In [ ]:
import hiplot as hip
from nnsysident.tables.bayesian import *

data = pd.DataFrame((TrainedModelBayesian() * ModelBayesian() & 'model_fn = "nnsysident.models.models.se2d_fullSXF"').fetch())
data = pd.concat([data, data['model_config'].apply(pd.Series)], axis = 1)

one_exp_h = data.set_index('score', drop=False).copy()
cols = ['score', 'gamma_readout', 'share_features', 'normalize' ]
hip.Experiment.from_dataframe(one_exp_h[cols]).display(force_full_width=False)